In [145]:
import pickle
import pandas as pd


In [147]:
class DataExtractor:
    def __init__(self, invoices, expired_invoices):
        self.invoices = invoices
        self.expired_invoices = expired_invoices

    def flat_data(self):
        items = []
        conversion_table = {0: "Material", 1: "Equipment", 2: "Service", 3: "Other"}

        for invoice in self.invoices:
            invoice_id = invoice["id"]
            if type(invoice["id"]) == str:
                invoice_id = invoice["id"].replace("O", "0") #there was 365371O(the last symbol is letter "O")
            try:
                created_on = pd.to_datetime(invoice["created_on"])
            except:
                created_on = pd.to_datetime("2024-06-05") #there was 30 february
            invoice_total = 0
            if "items" in invoice:
                for item in invoice["items"]:
                    if type(item["item"]["type"]) == str:
                        item["item"]["type"] = int(item["item"]["type"].replace("O", "0"))
                    if type(item["quantity"]) == str:
                        item["quantity"] = item["quantity"].replace("ten", "10")
                        item["quantity"] = item["quantity"].replace("five", "5")
                    item["quantity"] = int(item["quantity"])
                    invoice_total += item["item"]["unit_price"] * item["quantity"]  
                    
            if "items" in invoice:
                for item in invoice["items"]:
                    invoiceitem_id = item["item"]["id"]
                    invoiceitem_name = item["item"]["name"]
                    item_type = item["item"]["type"]
                    if type(item["item"]["type"]) == str:
                        item["item"]["type"] = int(item["item"]["type"].replace("O", "0"))
                    type_str = conversion_table[item["item"]["type"]]
                    unit_price = item["item"]["unit_price"]
                    # item_quantity = item["quantity"]
                    if type(item["quantity"]) == str:
                        item["quantity"] = item["quantity"].replace("ten", "10")
                        item["quantity"] = item["quantity"].replace("five", "5")
                    total_price = unit_price * int(item["quantity"])
                    percentage_in_invoice = total_price / invoice_total if invoice_total else 0
                    is_expired = invoice_id in self.expired_invoices

                    items.append([
                        invoice_id,
                        created_on,
                        invoiceitem_id,
                        invoiceitem_name,
                        type_str,
                        unit_price,
                        total_price,
                        percentage_in_invoice,
                        is_expired
                    ])

        columns = [
            "invoice_id", 
            "created_on", 
            "invoiceitem_id", 
            "invoiceitem_name", 
            "type", 
            "unit_price", 
            "total_price", 
            "percentage_in_invoice", 
            "is_expired"
        ]
        
        self.df = pd.DataFrame(items, columns=columns)
        self.df = self.df.astype({
            "invoice_id": "int",
            "created_on": "datetime64[ns]",
            "invoiceitem_id": "int",
            "invoiceitem_name": "str",
            "type": "str",
            "unit_price": "int",
            "total_price": "int",
            "percentage_in_invoice": "float",
            "is_expired": "bool"
        })
        self.df.sort_values(by=["invoice_id", "invoiceitem_id"], inplace=True)
    
    def get_dataframe(self):
        return self.df

invoices, expired_invoices = None, None
with open(invoices_file_path, "rb") as file:
    invoices = pickle.load(file)
    
with open(expired_file_path, "r") as file:
    expired_invoices = set(map(int, file.read().split(",")))

invoices_file_path = "invoices_new.pkl"
expired_file_path = "expired_invoices.txt"

extractor = DataExtractor(invoices, expired_invoices)
extractor.flat_data()
df = extractor.get_dataframe()
display(df)

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
292,301695,2019-04-26,103215,ii_103215,Other,135,945,0.374851,False
290,301695,2019-04-26,166227,ii_166227,Equipment,118,708,0.280841,False
293,301695,2019-04-26,171394,ii_171394,Other,128,128,0.050774,False
291,301695,2019-04-26,195625,ii_195625,Other,148,740,0.293534,False
130,304245,2019-03-17,121446,ii_121446,Service,158,632,0.128429,False
...,...,...,...,...,...,...,...,...,...
158,3852900,2019-06-29,168488,ii_168488,Equipment,116,232,0.225243,False
87,3977230,2019-05-11,117999,ii_117999,Equipment,196,392,0.212928,False
89,3977230,2019-05-11,121772,ii_121772,Material,120,600,0.325910,False
86,3977230,2019-05-11,181613,ii_181613,Material,161,483,0.262357,False


In [148]:
df.to_csv('new_data.csv', index=False)